In [1]:
#!pip install tensorflow==2.13.0
#!pip install chess pygame

In [2]:
import tensorflow as tf
import pygame
import chess
import math
import numpy as np

pygame 2.5.2 (SDL 2.28.3, Python 3.11.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
tf.__version__

'2.13.0'

In [4]:
def parse_fen(fen):
    #print(fen)
    #flipping the board for perspective
    file_to_num = {'a': 1, 'b': 2, 'c': 3, 'd':4, 'e':5, 'f':6, 'g':7, 'h':8}
    enc = np.zeros([8,8,18]).astype(int)
    fen_elem = fen.split(' ')
    fen_pos = fen_elem[0]
    if fen_elem[1] == 'w':
        player = 1
        enc_dict = {"R":0, "N":1, "B":2, "Q":3, "K":4, "P":5, "r":6, "n":7, "b":8, "q":9, "k":10, "p":11}
    else:
        player = 0
        enc_dict = {"r":0, "n":1, "b":2, "q":3, "k":4, "p":5, "R":6, "N":7, "B":8, "Q":9, "K":10, "P":11}
    enc[:,:,12] = player
    castle = fen_elem[2]
    if player:
        if 'Q' not in castle:
            enc[:,:,13] = 1
        if 'K' not in castle:
            enc[:,:,14] = 1
        if 'q' not in castle:
            enc[:,:,15] = 1
        if 'k' not in castle:
            enc[:,:,16] = 1
    else:
        if 'Q' not in castle:
            enc[:,:,15] = 1
        if 'K' not in castle:
            enc[:,:,16] = 1
        if 'q' not in castle:
            enc[:,:,13] = 1
        if 'k' not in castle:
            enc[:,:,14] = 1
    enc[:,:,17] = int(fen_elem[-1])
    ranks = fen_pos.split('/')
    for i, rank in enumerate(ranks):
        j = 0
        k = 0
        while k < len(rank):
            if rank[k].isdigit():
                j += int(rank[k])
                k += 1
                continue
            enc[(7 - i)*player + (1 - player)*i, j*player + (1 - player)*(7 - j), enc_dict[rank[k]]] = 1
            j += 1
            k += 1
    # appending the squares covered by pieces of both sides
    # l = 17
    # for j in range(2):
    #     pawns_sq = []
    #     if j == 1:
    #         fen = board.push(chess.Move.null())
    #     board = chess.Board(fen)
    #     legal_moves = board.legal_moves
    #     pieces = [enc_dict[str(board.piece_at(move.from_square))] for move in legal_moves]
    #     legal_moves_ordered = np.array(list(legal_moves))[np.argsort(pieces, kind = 'stable')]
    #     n = len(legal_moves_ordered)
    #     chg_dir = 1
    #     for i in range(n):
    #         move = legal_moves_ordered[i]
    #         move_str = str(move)
    #         initial_file = player*(file_to_num[move_str[0]] - 1) + (1 - player)*(8 - file_to_num[move_str[0]])
    #         initial_rank = player*(int(move_str[1]) - 1) + (1 - player)*(8 - int(move_str[1])) 
    #         from_sq = move.from_square
    #         if str(board.piece_at(from_sq)).upper() == 'P':
    #             if from_sq not in pawns_sq:
    #                 if str(board.piece_at(from_sq)) == 'P':
    #                     if initial_rank + 1 < 8 and initial_file - 1 >= 0:
    #                         enc[initial_rank + 1, initial_file - 1, l] = 1 - 2*j 
    #                     if initial_rank + 1 < 8 and initial_file + 1 < 8:
    #                         enc[initial_rank + 1, initial_file + 1, l] = 1 - 2*j
    #                 else:
    #                     if initial_rank - 1 >= 0 and initial_file - 1 >= 0:
    #                         enc[initial_rank - 1, initial_file - 1, l] = 1 - 2*j
    #                     if initial_rank - 1 >= 0 and initial_file + 1 < 8:
    #                         enc[initial_rank - 1, initial_file + 1, l] = 1 - 2*j
    #                 pawns_sq.append(from_sq)
    #                 l += 1
    #                 #print(str(move))
    #             continue
    #         if str(board.piece_at(from_sq)).upper() == 'N':
    #             if initial_file + 2 < 8 and initial_rank + 1 < 8:
    #                 enc[initial_rank + 1, initial_file + 2, l] = 1 - 2*j
    #             if initial_file + 2 < 8 and initial_rank - 1 >= 0:
    #                 enc[initial_rank - 1, initial_file + 2, l] = 1 - 2*j
    #             if initial_file + 1 < 8 and initial_rank + 2 < 8:
    #                 enc[initial_rank + 2, initial_file + 1, l] = 1 - 2*j
    #             if initial_file - 1 >= 0 and initial_rank + 2 < 8:
    #                 enc[initial_rank + 2, initial_file - 1, l] = 1 - 2*j
    #             if initial_file - 2 >= 0 and initial_rank + 1 < 8:
    #                 enc[initial_rank + 1, initial_file - 2, l] = 1 - 2*j
    #             if initial_file - 2 >= 0 and initial_rank - 1 >= 0:
    #                 enc[initial_rank - 1, initial_file - 2, l] = 1 - 2*j
    #             if initial_file - 1 >= 0 and initial_rank - 2 >= 0:
    #                 enc[initial_rank - 2, initial_file - 1, l] = 1 - 2*j
    #             if initial_file + 1 < 8 and initial_rank - 2 >= 0:
    #                 enc[initial_rank - 2, initial_file + 1, l] = 1 - 2*j
    #             while i + 1 < n and legal_moves_ordered[i + 1].from_square == from_sq:
    #                 i += 1
    #             l += 1
    #             #print(str(move))
    #             continue
    #         final_file = player*(file_to_num[move_str[2]] - 1) + (1 - player)*(8 - file_to_num[move_str[2]])
    #         final_rank = player*(int(move_str[3]) - 1) + (1 - player)*(8 - int(move_str[3]))
    #         if chg_dir:
    #             if not board.is_capture(move) and 0 <= final_rank + np.sign(final_rank - initial_rank) < 8 and 0 <= final_file + np.sign(final_file - initial_file) < 8:
    #                 enc[final_rank + np.sign(final_rank - initial_rank), final_file + np.sign(final_file - initial_file), l] = 1 - 2*j
    #             chg_dir = 0
    #         elif abs(final_rank - initial_rank) == 1 or abs(final_file - initial_file) == 1:
    #             chg_dir = 1
    #         enc[final_rank, final_file, l] = 1 - 2*j
    #         if i + 1 < n and legal_moves_ordered[i + 1].from_square != from_sq:
    #             w_sq = (initial_rank, initial_file - 1)
    #             e_sq = (initial_rank, initial_file + 1)
    #             n_sq = (initial_rank + 1, initial_file)
    #             s_sq = (initial_rank - 1, initial_file)
    #             ne_sq = (initial_rank + 1, initial_file + 1)
    #             nw_sq = (initial_rank + 1, initial_file - 1)
    #             se_sq = (initial_rank - 1, initial_file + 1)
    #             sw_sq = (initial_rank - 1, initial_file - 1)
    #             if str(board.piece_at(from_sq)).upper():
    #                 if w_sq[1] >= 0:
    #                     enc[w_sq[0], w_sq[1], l] = 1 - 2*j
    #                 if e_sq[1] < 8:
    #                     enc[e_sq[0], e_sq[1], l] = 1 - 2*j
    #                 if n_sq[0] < 8:
    #                     enc[n_sq[0], n_sq[1], l] = 1 - 2*j
    #                 if s_sq[0] >= 0:
    #                     enc[s_sq[0], s_sq[1], l] = 1 - 2*j
    #             elif str(board.piece_at(from_sq)).upper():
    #                 if ne_sq[0] < 8 and ne_sq[1] < 8:
    #                     enc[ne_sq[0], ne_sq[1], l] = 1 - 2*j
    #                 if nw_sq[0] < 8 and nw_sq[1] >= 0:
    #                     enc[nw_sq[0], nw_sq[1], l] = 1 - 2*j
    #                 if se_sq[0] >= 0 and se_sq[1] < 8:
    #                     enc[se_sq[0], se_sq[1], l] = 1 - 2*j
    #                 if sw_sq[0] >= 0 and sw_sq[1] >= 0:
    #                     enc[sw_sq[0], sw_sq[1], l] = 1 - 2*j
    #             else: #for king and queen all directions
    #                 if w_sq[1] >= 0:
    #                     enc[w_sq[0], w_sq[1], l] = 1 - 2*j
    #                 if e_sq[1] < 8:
    #                     enc[e_sq[0], e_sq[1], l] = 1 - 2*j
    #                 if n_sq[0] < 8:
    #                     enc[n_sq[0], n_sq[1], l] = 1 - 2*j
    #                 if s_sq[0] >= 0:
    #                     enc[s_sq[0], s_sq[1], l] = 1 - 2*j
    #                 if ne_sq[0] < 8 and ne_sq[1] < 8:
    #                     enc[ne_sq[0], ne_sq[1], l] = 1 - 2*j
    #                 if nw_sq[0] < 8 and nw_sq[1] >= 0:
    #                     enc[nw_sq[0], nw_sq[1], l] = 1 - 2*j
    #                 if se_sq[0] >= 0 and se_sq[1] < 8:
    #                     enc[se_sq[0], se_sq[1], l] = 1 - 2*j
    #                 if sw_sq[0] >= 0 and sw_sq[1] >= 0:
    #                     enc[sw_sq[0], sw_sq[1], l] = 1 - 2*j
    #             l += 1        
    #             #print(str(move))
    return enc

In [5]:
def parse_actions(move, fen):
    if fen.split(' ')[1] == 'w':
        player = 1
    else:
        player = 0
    enc = np.zeros([8,8,73]).astype(int)
    file_to_num = {'a': 1, 'b': 2, 'c': 3, 'd':4, 'e':5, 'f':6, 'g':7, 'h':8}
    init_file = file_to_num[move[0]]
    init_rank = int(move[1])
    final_file = file_to_num[move[2]]
    final_rank = int(move[3])
    under_promo = move[-1] if move[-1] in ['n', 'N', 'b', 'B', 'r', 'R'] else None
    file_diff = (init_file - final_file)*player + (final_file - init_file)*(1 - player)
    rank_diff = (init_rank - final_rank)*player + (final_rank - init_rank)*(1 - player)
    if under_promo is not None: #underpromotions
        if file_diff == 0: #no capture
            if under_promo in ['r', 'R']:
                idx = 64
            elif under_promo in ['b', 'B']:
                idx = 65
            else:
                idx = 66
        elif file_diff > 0: # left capture
            if under_promo in ['r', 'R']:
                idx = 67
            elif under_promo in ['b', 'B']:
                idx = 68
            else:
                idx = 69
        else: # right capture
            if under_promo in ['r', 'R']:
                idx = 70
            elif under_promo in ['b', 'B']:
                idx = 71
            else:
                idx = 72
    elif file_diff == 0: #NS direction
        if rank_diff < 0: #upward
            idx = -rank_diff - 1
        else: #downward
            idx = 6 + rank_diff
    elif rank_diff == 0: #EW direction
        if file_diff < 0: #right
            idx = 13 - file_diff
        else: #left
            idx = 20 + file_diff
    elif abs(file_diff) == abs(rank_diff): #diagonal moves
        if rank_diff < 0 and file_diff < 0: #NE
            idx = 27 - rank_diff
        elif rank_diff > 0 and file_diff > 0: #SW
            idx = 34 + rank_diff
        elif rank_diff < 0 and file_diff > 0: #NW
            idx = 41 + file_diff
        else: #SE
            idx = 48 + rank_diff
    elif file_diff == 1 and rank_diff == -2:
        idx = 56
    elif file_diff == -1 and rank_diff == -2:
        idx = 57
    elif file_diff == 2 and rank_diff == 1:
        idx = 58
    elif file_diff == 2 and rank_diff == -1:
        idx = 59
    elif file_diff == 1 and rank_diff == 2:
        idx = 60
    elif file_diff == -1 and rank_diff == 2:
        idx = 61
    elif file_diff == -2 and rank_diff == 1:
        idx = 62
    elif file_diff == -2 and rank_diff == -1:
        idx = 63
    enc[(init_rank - 1)*player + (8 - init_rank)*(1 - player), (init_file - 1)*player + (8 - init_file)*(1 - player), idx] = 1
    return enc.flatten(), (init_file - 1)*player + (8 - init_file)*(1 - player), (init_rank - 1)*player + (8 - init_rank)*(1 - player), idx

In [6]:
@tf.keras.utils.register_keras_serializable()
class ConvBlock(tf.keras.layers.Layer):
    def __init__(self, filters, kernel_size, **kwargs):
        super(ConvBlock, self).__init__(**kwargs)
        self.conv = tf.keras.layers.Conv2D(filters, kernel_size, padding = 'same')
        self.batch_norm = tf.keras.layers.BatchNormalization()
        self.relu = tf.keras.layers.ReLU()

    def call(self, inputs):
        x = self.conv(inputs)
        x = self.batch_norm(x)
        x = self.relu(x)
        return x

@tf.keras.utils.register_keras_serializable()
class ResBlock(tf.keras.layers.Layer):
    def __init__(self, filters, kernel_size, **kwargs):
        super(ResBlock, self).__init__(**kwargs)
        self.conv1 = tf.keras.layers.Conv2D(filters, kernel_size, padding = 'same')
        self.batch_norm1 = tf.keras.layers.BatchNormalization()
        self.relu = tf.keras.layers.ReLU()
        self.conv2 = tf.keras.layers.Conv2D(filters, kernel_size, padding = 'same')
        self.batch_norm2 = tf.keras.layers.BatchNormalization()

    def call(self, inputs):
        out = self.conv1(inputs)
        out = self.batch_norm1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.batch_norm2(out)
        return self.relu(out + inputs)
    
@tf.keras.utils.register_keras_serializable()
class PolicyHead(tf.keras.layers.Layer):
    def __init__(self, filters, kernel_size, num_move_planes, **kwargs):
        super(PolicyHead, self).__init__(**kwargs)
        self.conv = tf.keras.layers.Conv2D(filters, kernel_size, padding = 'same')
        self.batch_norm = tf.keras.layers.BatchNormalization()
        self.relu = tf.keras.layers.ReLU()
        self.flatten = tf.keras.layers.Flatten()
        self.dense = tf.keras.layers.Dense(num_move_planes * 8 * 8, activation = 'softmax')

    def call(self, inputs):
        x = self.conv(inputs)
        x = self.batch_norm(x)
        x = self.relu(x)
        x = self.flatten(x)
        x = self.dense(x)
        return x

In [7]:
def decode_action(out_policy, fen):
    board = chess.Board(fen)
    legal_moves = list(board.legal_moves)
    legal_moves_str = np.array([str(move) for move in legal_moves])
    prob = np.zeros(len(legal_moves))
    for i, move in enumerate(legal_moves_str):
        _, x, y, z = parse_actions(move, fen) #x = file, y = rank
        prob[i] = out_policy[y, x, z]
    idx = prob.argmax()
    best_move = legal_moves[idx]
    return best_move

In [8]:
model = tf.keras.models.load_model('C:/Users/DELL/Downloads/model_res10.h5')

In [9]:
def agent(BOARD, model):
    fen = BOARD.fen()
    enc_fen = parse_fen(fen)
    out_policy = model.predict(np.array([enc_fen]))[0].reshape((8, 8, 73))
    return decode_action(out_policy, fen)

In [10]:
def agent_hist(model, move_hist, history = 3):
    stack_height = 18*(history + 1)
    stack_input = np.empty((8, 8, stack_height))
    for j in range(history, -1, -1):
        if j < len(move_hist):
            stack_input[:, :, 18*(history - j):18*(history + 1 - j)] = parse_fen(move_hist[-1 - j])
        else:
            stack_input[:, :, 18*(history - j):18*(history + 1 - j)] = parse_fen(move_hist[0]) #repeating the start position until moves_played > j
    out_policy = model.predict(np.array([stack_input]))[0].reshape((8, 8, 73))
    return decode_action(out_policy, move_hist[-1])

In [16]:
#initialise display
X = 800
Y = 800
scrn = pygame.display.set_mode((X, Y))
pygame.init()

#basic colours
WHITE = (255, 255, 255)
GREY = (128, 128, 128)
YELLOW = (204, 204, 0)
BLUE = (50, 255, 255)
BLACK = (0, 0, 0)

#initialise chess board
b = chess.Board()

#load piece images
pieces = {'p': pygame.image.load('pieces/b_pawn.png'),
          'n': pygame.image.load('pieces/b_knight.png'),
          'b': pygame.image.load('pieces/b_bishop.png'),
          'r': pygame.image.load('pieces/b_rook.png'),
          'q': pygame.image.load('pieces/b_queen.png'),
          'k': pygame.image.load('pieces/b_king.png'),
          'P': pygame.image.load('pieces/w_pawn.png'),
          'N': pygame.image.load('pieces/w_knight.png'),
          'B': pygame.image.load('pieces/w_bishop.png'),
          'R': pygame.image.load('pieces/w_rook.png'),
          'Q': pygame.image.load('pieces/w_queen.png'),
          'K': pygame.image.load('pieces/w_king.png')
          }
#load the chess board
board_img = pygame.image.load('board.png')

def update(scrn,board):
    '''
    updates the screen basis the board class
    '''
    
    for i in range(64):
        piece = board.piece_at(i)
        if piece == None:
            pass
        else:
            scrn.blit(pieces[str(piece)],((i%8)*100,700-(i//8)*100))
    
    for i in range(7):
        i=i+1
        pygame.draw.line(scrn,WHITE,(0,i*100),(800,i*100))
        pygame.draw.line(scrn,WHITE,(i*100,0),(i*100,800))

    pygame.display.flip()
    
def main_one_agent(agent,model,agent_color=True,BOARD=chess.Board()):
    
    '''
    for agent vs human game
    color is True = White agent
    color is False = Black agent
    '''
    #name window
    if agent_color:
        pygame.display.set_caption('You are Black!')
    else:
        pygame.display.set_caption('You are White!')
    
    #variable to be used later
    index_moves = []
    move_hist = [] #history of moves played by the agent
    status = True
    scrn.blit(board_img, (0, 0))
    update(scrn, BOARD)
    while (status):
        if BOARD.turn==agent_color:
            move_hist.append(BOARD.fen())
            # BOARD.push(agent_hist(model,move_hist,history=3))
            BOARD.push(agent(BOARD, model))
            #scrn.fill(BLUE)
            scrn.blit(board_img, (0, 0))
            update(scrn, BOARD)
        else:
            for event in pygame.event.get():
                # if event object type is QUIT
                # then quitting the pygame
                # and program both.
                if event.type == pygame.QUIT:
                    status = False
                # if mouse clicked
                if event.type == pygame.MOUSEBUTTONDOWN:
                    #reset previous screen from clicks
                    #scrn.fill(BLUE)
                    scrn.blit(board_img, (0, 0))
                    update(scrn, BOARD)
                    #get position of mouse
                    pos = pygame.mouse.get_pos()
                    #find which square was clicked and index of it
                    square = (math.floor(pos[0]/100),math.floor(pos[1]/100))
                    index = (7-square[1])*8+(square[0])
                    # if we have already highlighted moves and are making a move
                    if index in index_moves: 
                        move = moves[index_moves.index(index)]
                        #print(BOARD)
                        #print(move)
                        BOARD.push(move)
                        scrn.blit(board_img, (0, 0))
                        update(scrn, BOARD)
                        index=None
                        index_moves = []
                    # show possible moves
                    else:
                        piece = BOARD.piece_at(index)
                        if piece == None:
                            pass
                        else:
                            all_moves = list(BOARD.legal_moves)
                            moves = []
                            for m in all_moves:
                                if m.from_square == index:
                                    moves.append(m)
                                    t = m.to_square
                                    TX1 = 100*(t%8)
                                    TY1 = 100*(7-t//8)
                                    pygame.draw.rect(scrn,YELLOW,pygame.Rect(TX1,TY1,100,100),5)
                                    pygame.display.flip()
                            #print(moves)
                            index_moves = [a.to_square for a in moves]
    # deactivates the pygame library
        if BOARD.outcome() != None:
            print(BOARD.outcome())
            status = False
            print(BOARD)
    pygame.quit()
    
main_one_agent(agent, model,agent_color=False)

1/1 [==============================] - 0s 38ms/step
